In [1]:
import pandas as pd
import requests
import json
import datetime, dateutil
import concurrent.futures
import swifter
import os
import mariadb
import sys

In [2]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

In [3]:
def conectar_banco():    
    try:
        conn = mariadb.connect(
            user="root",
            password="abc@123",
            host="172.17.0.3",
            port=3306,
            database='dados_tribunais',
            autocommit=True
        )
        print("Connection established successfully!")
    except mariadb.Error as e:
        print(f"Error connecting to MariaDB: {e}")
        
    # Get Cursor
    cursor = conn.cursor()
    return cursor

#     processos.append([numero_processo, classe, data_ajuizamento, ultima_atualizacao, \
#                      codigo, orgao_julgador, municipio, grau, assuntos, movimentos, sort])

def criar_bd(nome):
    cursor = conectar_banco()
    cursor.execute("CREATE DATABASE IF NOT EXISTS dados_tribunais")
    #cursor.commit()
    cursor.close()
    
def criar_tabela():
    try:
        cursor = conectar_banco()

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS processos (
                numero_processo VARCHAR(255) PRIMARY KEY,
                tribunal VARCHAR(255),
                classe VARCHAR(255),
                data_ajuizamento DATE,
                ultima_atualizacao DATE,
                codigo VARCHAR(50),
                orgao_julgador VARCHAR(255),
                municipio VARCHAR(255),
                grau VARCHAR(50),
                assuntos TEXT,
                movimentos TEXT
            )
        ''')

        print("Tabela 'tribunal_data' criada com sucesso!")

    except: 
        print(f"Erro na criação da tabela")
    cursor.close()

            
def adicionar_processo_mariadb(cursor, numero_processo, tribunal, classe, data_ajuizamento, ultima_atualizacao, codigo, orgao_julgador, municipio, grau, assuntos, movimentos):
    try:
        cursor.execute('''
            INSERT INTO processos (numero_processo, tribunal, classe, data_ajuizamento, ultima_atualizacao, codigo, orgao_julgador, municipio, grau, assuntos, movimentos)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ''', (numero_processo, tribunal, classe, data_ajuizamento, ultima_atualizacao, codigo, orgao_julgador, municipio, grau, assuntos, movimentos))
        
        
    except mariadb.Error as e:
        print(f"Error adding entry to database: {e}")

                    

In [4]:
def converte_data(data_str):
    data = dateutil.parser.parse(data_str)
    data = datetime.datetime.date(data)
    return data.strftime('%Y-%m-%d')


def gera_lista_assuntos(assuntos_do_df):
    lst_assuntos=[]
    for assunto in assuntos_do_df:
        try:
            lst_assuntos.append(assunto.get('nome'))
        except:
            lst_assuntos.append('')

    return lst_assuntos


def gera_lista_movimentos(movimentos):
    lst_movimentos_final =[]
    for movimento in movimentos:
        codigo = movimento.get('codigo')
        nome = movimento.get('nome')
        data_hora = movimento.get('dataHora')
        if data_hora:
            data_hora = converte_data(data_hora)
        lst_movimentos_final.append([ codigo, nome, data_hora])
    return lst_movimentos_final

def process_movimento(movimento):
    codigo = movimento.get('codigo')
    nome = movimento.get('nome')
    data_hora = movimento.get('dataHora')
    if data_hora:
        data_hora = converte_data(data_hora)
    return [codigo, nome, data_hora]

def gera_lista_movimentos_multithread(movimentos):
    lst_movimentos_final = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(process_movimento, movimentos))
        lst_movimentos_final.extend(results)
    return lst_movimentos_final

In [5]:
def obter_data(item):
    return datetime.strptime(item[2], '%Y-%m-%d')
  
def lista_para_dataframe(conector, dados_dict):
  processos = []
  for processo in dados_dict['hits']['hits']:
    numero_processo = processo['_source']['numeroProcesso']
    grau = processo['_source']['grau']
    tribunal = processo['_source']['tribunal']
    classe = processo['_source']['classe']['nome']
    try:
      assuntos = processo['_source']['assuntos'] # Pode ter mais de um
      asuntos = gera_lista_assuntos(assuntos)
    except:
      assuntos = []
    data_ajuizamento = converte_data(processo['_source']['dataAjuizamento'])
    ultima_atualizacao = converte_data(processo['_source']['dataHoraUltimaAtualizacao'])
    #formato = processo['_source']['formato']['nome']
    codigo = processo['_source']['orgaoJulgador']['codigo']
    orgao_julgador = processo['_source']['orgaoJulgador']['nome']
    municipio = processo['_source']['orgaoJulgador']['codigoMunicipioIBGE']
    sort = processo['sort'][0]
    try:
      movimentos = processo['_source']['movimentos']
      movimentos = gera_lista_movimentos_multithread(movimentos)
     
      movimentos = sorted(movimentos, key=lambda x: x[2])
      #print(movimentos)     
    except:
      movimentos = []
    assuntos_str = '; '.join(str(assunto) for assunto in assuntos)
    movimentos_str = '; '.join(str(movimento) for movimento in movimentos)
    adicionar_processo_mariadb( conector, numero_processo, tribunal, classe,
                               data_ajuizamento, ultima_atualizacao, codigo,
                               orgao_julgador, municipio, grau, assuntos_str,
                               movimentos_str)  

In [6]:
def criar_dataset(conector, codigo_orgao, data, tamanho_consulta):
  
  df_tribunal = pd.DataFrame()
  url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjrn/_search"
  api_key = "APIKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw==" # Chave pública
  tribunal = 'TJRN'
  orgaoJulgador = codigo_orgao
  size = tamanho_consulta
  data = data

  payload = json.dumps(
  {
  "size": tamanho_consulta,
  "query": {
      "bool": {
        "must": [
            {"match": {"tribunal": tribunal}},
            {"match": {"orgaoJulgador.codigo": orgaoJulgador}},
            {"range": {"dataAjuizamento": {"gte": data }}}
        ]
      }
  },
    "sort": [{"@timestamp": {"order": "asc"}}]
  })

  headers = {
    'Authorization': api_key,
    'Content-Type': 'application/json'
  }

  response = requests.request("POST", url, headers=headers, data=payload)  # <Response [200]>
  dados_dict = response.json() # <class 'dict'>
  if len(dados_dict['hits']['hits']) < 5:
    print(f'Parece que você digitou um código de Serventia errado. Confira novamente.')
  lista_para_dataframe(conector, dados_dict)
  numero_processos = size

  while numero_processos == size:
    numero_processos = len(dados_dict['hits']['hits'])
    tamanho_dicionario_retornado = len(dados_dict['hits']['hits'])-1
    if tamanho_dicionario_retornado < 1:
      print(f'Tamanho do dicionário da página anterior: {tamanho_dicionario_retornado}')
      continue
    ultima_posicao_dicionario = dados_dict['hits']['hits'][(len(dados_dict['hits']['hits'])-1)]['sort'][0]
    #print(f'Partindo da posição: {ultima_posicao_dicionario}')
    payload = json.dumps(
    {
    "size": tamanho_consulta,
    "query": {
        "bool": {
          "must": [
            {"match": {"tribunal": tribunal}},
            {"match": {"orgaoJulgador.codigo": orgaoJulgador}},
            {"range": {"dataAjuizamento": {"gte": data}}}
            
          ]
        }
    },
      "search_after": [ ultima_posicao_dicionario ],
      "sort": [{"@timestamp": {"order": "asc"}}]
    })

    headers = {
      'Authorization': api_key,
      'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)  # <Response [200]>
    dados_dict = response.json() # <class 'dict'>
    numero_processos = len(dados_dict['hits']['hits'])
    ultima_posicao_dicionario = dados_dict['hits']['hits'][(len(dados_dict['hits']['hits'])-1)]['sort']
    lista_para_dataframe(conector, dados_dict)
    ultima_data_ajuizamento = dados_dict['hits']['hits'][len(dados_dict['hits']['hits'])-1]['_source']['dataAjuizamento']
    orgao_julgador = dados_dict['hits']['hits'][len(dados_dict['hits']['hits'])-1]['_source']['orgaoJulgador']['nome']
    print(f'{datetime.datetime.now()}\t Serventia: {orgao_julgador}\t Data do último processo adicionado: {ultima_data_ajuizamento}' )
    #if tamanho_dataset > 2000000:
    #  break

  return

In [7]:
def salvar_dataset(df, orgao, data):        
    if not type(orgao) is str:
        str(orgao)
    orgao = orgao.replace(' ', '_')
    nome_df = 'TJRN_' + orgao + '_' + data + '.csv'
    nome_df = nome_df.replace(" ", "_")
    if os.path.isdir('./dados'):
        os.chdir('dados')
    df.to_csv(nome_df, sep=';', header=True, index=False)

In [8]:
lista_codigo_orgaos = [75496, 17460, 17461 ] # 4º, 2º e 1º JEFP
data = '2018-01-01'
criar_bd('dados_tribunais')
criar_tabela()
cursor = conectar_banco()
for orgao in lista_codigo_orgaos:
    criar_dataset(cursor, orgao, data, 10000)

cursor.close()

Connection established successfully!
Connection established successfully!
Tabela 'tribunal_data' criada com sucesso!
Connection established successfully!
Error adding entry to database: Duplicate entry '08281768720228205001' for key 'PRIMARY'
Error adding entry to database: Duplicate entry '08071280420248205001' for key 'PRIMARY'
Error adding entry to database: Duplicate entry '08051921220228205001' for key 'PRIMARY'
Error adding entry to database: Duplicate entry '08457789620198205001' for key 'PRIMARY'
Error adding entry to database: Duplicate entry '08725191320188205001' for key 'PRIMARY'
Error adding entry to database: Duplicate entry '08141444820208205001' for key 'PRIMARY'
Error adding entry to database: Duplicate entry '08070528720188205001' for key 'PRIMARY'
Error adding entry to database: Duplicate entry '08605996620238205001' for key 'PRIMARY'
Error adding entry to database: Duplicate entry '08628531720208205001' for key 'PRIMARY'
Error adding entry to database: Duplicate ent